In [1]:
# Model 3
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from math import radians, sin, cos, sqrt, atan2

In [3]:
from google.colab import files
uploaded = files.upload()
df3 = pd.read_csv("dataset.csv")
df3.head()

Saving dataset.csv to dataset.csv


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [4]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Radius of Earth in meters
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c  # in meters# Haversine distance function


In [5]:
# Doing the same as did previously in Model 2

df3["Timestamp"] = pd.to_datetime(df3["LastUpdatedDate"] + " " + df3["LastUpdatedTime"], format='%d-%m-%Y %H:%M:%S')
df3.sort_values(["SystemCodeNumber", "Timestamp"], inplace=True)


# Demand Calculation (Model 3)
vehicle_weights = {"car": 1.0, "bike": 0.5, "truck": 1.5}
traffic_levels = {"low": 1.0, "medium": 0.5, "high": 0.1}
df3["VehicleWeight"] = df3["VehicleType"].map(vehicle_weights)
df3["TrafficScore"] = df3["TrafficConditionNearby"].map(traffic_levels)



α, β, γ, δ, ε = 2.0, 1.0, 1.5, 2.0, 1.0
df3["DemandRaw"] = (
    α * (df3["Occupancy"] / df3["Capacity"]) +
    β * df3["QueueLength"].fillna(0) -
    γ * df3["TrafficScore"].fillna(1.0) +
    δ * df3["IsSpecialDay"] +
    ε * df3["VehicleWeight"].fillna(1.0)
)

In [6]:
# Normalising demand per timestamp
scaler = MinMaxScaler()
df3["DemandNorm"] = scaler.fit_transform(df3[["DemandRaw"]])

In [7]:
# Base Price Calculation
BASE_PRICE = 10
LAMBDA = 0.5
MIN_PRICE = 5
MAX_PRICE = 20
df3["BasePrice"] = BASE_PRICE * (1 + LAMBDA * df3["DemandNorm"])
df3["BasePrice"] = df3["BasePrice"].clip(MIN_PRICE, MAX_PRICE)

In [8]:
# Apply Competitive Adjustment
theta = 0.1  # Adjustment factor
adjusted_prices = []

# For each row, find nearby competitors and adjust
for idx, row in df3.iterrows():
    lat1, lon1 = row["Latitude"], row["Longitude"]
    time = row["Timestamp"]

    # Filter same timestamp rows
    same_time_df3 = df3[df3["Timestamp"] == time]

    # Find nearby lots (excluding itself)
    nearby = same_time_df3[
        same_time_df3["SystemCodeNumber"] != row["SystemCodeNumber"]
    ].copy()

    # Calculate distance to all others
    nearby["Distance"] = nearby.apply(
        lambda x: haversine(lat1, lon1, x["Latitude"], x["Longitude"]), axis=1
    )

    nearby = nearby[nearby["Distance"] <= 500]

    if not nearby.empty:
        avg_price = nearby["BasePrice"].mean()
        price_adj = row["BasePrice"] + theta * (row["BasePrice"] - avg_price)
    else:
        price_adj = row["BasePrice"]

    adjusted_prices.append(price_adj)

df3["AdjustedPrice"] = np.clip(adjusted_prices, MIN_PRICE, MAX_PRICE)

# ✅ Done!
print(df3[["SystemCodeNumber", "Timestamp", "BasePrice", "AdjustedPrice"]].head(10))

  SystemCodeNumber           Timestamp  BasePrice  AdjustedPrice
0      BHMBCCMKT01 2016-10-04 07:59:00  10.406343      10.372134
1      BHMBCCMKT01 2016-10-04 08:25:00  10.408868      10.373782
2      BHMBCCMKT01 2016-10-04 08:59:00  10.665173      10.652455
3      BHMBCCMKT01 2016-10-04 09:32:00  10.687900      10.671305
4      BHMBCCMKT01 2016-10-04 09:59:00  10.602675      10.573542
5      BHMBCCMKT01 2016-10-04 10:26:00  10.989659      10.971426
6      BHMBCCMKT01 2016-10-04 10:59:00  12.202774      12.276047
7      BHMBCCMKT01 2016-10-04 11:25:00  11.534255      11.526526
8      BHMBCCMKT01 2016-10-04 11:59:00  11.544356      11.524491
9      BHMBCCMKT01 2016-10-04 12:29:00  12.485173      12.453505


In [10]:
from pyngrok import ngrok
ngrok.set_auth_token("2zXeD5U43qMqAbY9iIIh57guWE1_4C1NfrnGHuEMnKfdYKfv2")

In [11]:
bokeh_code = '''
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, DatetimeTickFormatter
from bokeh.plotting import figure, curdoc
from bokeh.driving import linear
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
from sklearn.preprocessing import MinMaxScaler

# Simulate example data
n_points = 100
timestamps = pd.date_range(start='2025-07-07 08:00', periods=n_points, freq='30min')

occupancy = np.random.randint(10, 50, size=n_points)
capacity = 50
queue = np.random.randint(0, 10, size=n_points)
traffic = np.random.choice([1.0, 0.5, 0.1], size=n_points)
special_day = np.random.choice([0, 1], size=n_points)
vehicle_weight = np.random.choice([1.0, 0.5, 1.5], size=n_points)

# Demand formula
α, β, γ, δ, ε = 2.0, 1.0, 1.5, 2.0, 1.0
demand_raw = (
    α * (occupancy / capacity)
    + β * queue
    - γ * traffic
    + δ * special_day
    + ε * vehicle_weight
)

scaler = MinMaxScaler()
demand_norm = scaler.fit_transform(demand_raw.reshape(-1, 1)).flatten()

BASE_PRICE = 10
LAMBDA = 0.5
base_prices = BASE_PRICE * (1 + LAMBDA * demand_norm)
base_prices = np.clip(base_prices, 5, 20)

# Simulated competitor prices
competitor_prices = base_prices + np.random.normal(loc=0, scale=1.0, size=n_points)
competitor_prices = np.clip(competitor_prices, 5, 20)

# Competitive adjustment
theta = 0.1
adjusted_prices = base_prices + theta * (base_prices - competitor_prices)
adjusted_prices = np.clip(adjusted_prices, 5, 20)

# Bokeh plotting
source = ColumnDataSource(data=dict(time=[], your_price=[], comp_price=[]))

p = figure(title="Model 3 - Real-Time Competitive Pricing",
           x_axis_type='datetime', width=800, height=400)
p.line(x='time', y='your_price', source=source, color='green', line_width=2, legend_label='Your Price')
p.line(x='time', y='comp_price', source=source, color='orange', line_width=2, legend_label='Competitor Avg')
p.xaxis.axis_label = "Time"
p.yaxis.axis_label = "Price ($)"
p.xaxis.formatter = DatetimeTickFormatter(minutes="%H:%M", hours="%H:%M")
p.legend.location = "top_left"

index = dict(i=0)

@linear()
def update(step):
    i = index['i']
    if i >= len(timestamps):
        return
    new_data = dict(
        time=[timestamps[i]],
        your_price=[adjusted_prices[i]],
        comp_price=[competitor_prices[i]]
    )
    source.stream(new_data, rollover=200)
    index['i'] += 1

curdoc().add_root(column(p))
curdoc().add_periodic_callback(update, 500)
'''

# Write to file
with open("model3_bokeh.py", "w") as f:
    f.write(bokeh_code)

In [ ]:
# Start new tunnel
public_url = ngrok.connect(5006)
print("🌐 Open this Bokeh App URL:", public_url )

# Run Bokeh server in background
!bokeh serve --allow-websocket-origin="*" model3_bokeh.py --port=5006 &

🌐 Open this Bokeh App URL: NgrokTunnel: "https://80bf03139597.ngrok-free.app" -> "http://localhost:5006"
2025-07-08 07:08:37,836 Starting Bokeh server version 3.7.3 (running on Tornado 6.4.2)
2025-07-08 07:08:37,837 Host wildcard '*' will allow connections originating from multiple (or possibly all) hostnames or IPs. Use non-wildcard values to restrict access explicitly
2025-07-08 07:08:37,837 User authentication hooks NOT provided (default user enabled)
2025-07-08 07:08:37,841 Bokeh app running at: http://localhost:5006/model3_bokeh
2025-07-08 07:08:37,841 Starting Bokeh server with process id: 4873
2025-07-08 07:08:47,820 WebSocket connection opened
2025-07-08 07:08:47,821 ServerConnection created
2025-07-08 07:08:51,073 WebSocket connection closed: code=1001, reason=None
